In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from collections import Counter  
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
import incertidumbre


In [2]:
#Extraccion y limpieza de datos.

datos_diabetes = pd.read_csv('/home/juan/machineLearning2025/datasets/diabetes_012_health_indicators_BRFSS2015.csv')

datos_diabetes['diabetes_01'] = datos_diabetes['Diabetes_012']
datos_diabetes['diabetes_01'] = datos_diabetes['diabetes_01'].replace(2,1)

#Reparar nombres de columnas. Se usa el formato loweCamelCase para el nombre de las caracteristicas.

new_col_names = []

for name in datos_diabetes.columns:
    # Luego, pon todas las letras en minúsculas
    name_lowered_first_letter = name[0].lower() + name[1:]
    # Elimina los espacios al principio y al final
    name_stripped = name_lowered_first_letter.strip()
    # Por último, reemplaza los espacios entre palabras por guiones bajos
    name_no_spaces = name_stripped.replace(' ', '_')
    # Agrega el nuevo nombre a la lista de nuevos nombres de columna
    new_col_names.append(name_no_spaces)

datos_diabetes.columns = new_col_names

datos_diabetes = datos_diabetes.rename(columns={'bMI':'bmi'})

#Definicion de datos enteros.
for col in datos_diabetes.columns:
    #if datos_diabetes[col].dtype == 'float64':
    datos_diabetes[col] = datos_diabetes[col].astype(int)

In [3]:
X = datos_diabetes.drop(["diabetes_01", "diabetes_012"], axis=1)
y = datos_diabetes["diabetes_01"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42
)
os_us3 = SMOTE(sampling_strategy=0.6, random_state=4321)
X_train_res_smote2, y_train = os_us3.fit_resample(X_train, y_train)
X_train =X_train_res_smote2.values
X_test = X_test.values

In [4]:
#usare codificacion binaria para los comosomas.

num_features =21

#Inicializacion binaria de un comosoma
cromosoma = np.random.randint(2, size=num_features)

cromosoma

array([0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1])

In [5]:

import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

def fitness_function(features):
    if np.sum(features) == 0:
        return 0.0

    features_mask = features.astype(bool)
    
    X_train_subset = X_train[:, features_mask]
    X_test_subset = X_test[:, features_mask]
    
    if X_train_subset.shape[1] == 0:
        return 0.0

    model = LogisticRegression()
    model.fit(X_train_subset, y_train)
    
    y_pred = model.predict(X_test_subset)
    f1 = accuracy_score(y_test, y_pred)
    
    print(f"Probando con {np.sum(features)} características. F1-score obtenido: {f1}")
    return f1
#Funciones para los Operadores Geneticos.

#seleccion

def selection(population, fitness_values):
    # Roulette wheel selection
    probabilities = fitness_values / np.sum(fitness_values)
    selected_indices = np.random.choice(len(population), size=len(population), p=probabilities)
    return population[selected_indices]

def crossover(parents):
    # Single-point crossover
    crossover_point = np.random.randint(1, len(parents[0]))
    offspring = np.empty_like(parents)
    for i in range(len(parents)):
        parent1, parent2 = parents[np.random.choice(len(parents), size=2, replace=False)]
        offspring[i] = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    return offspring

def mutation(offspring, mutation_rate):
    # Bit-wise mutation
    for i in range(len(offspring)):
        for j in range(len(offspring[i])):
            if np.random.rand() < mutation_rate:
                offspring[i][j] = 1 - offspring[i][j]
    return offspring


In [6]:
# Genetic Algorithm Framework Example
class GeneticAlgorithm:
    def __init__(self, population_size, chromosome_length, fitness_function):
        self.population_size = population_size
        self.chromosome_length = chromosome_length
        self.fitness_function = fitness_function
        self.population = self.initialize_population()
        self.best_chromosome_so_far = None
        self.best_fitness_so_far = -1 # Inicia con un valor muy bajo
    
    def initialize_population(self):
        # Initialize population randomly
        return np.random.randint(2, size=(self.population_size, self.chromosome_length))
    def evolve(self, num_generations, mutation_rate):
        for generation in range(num_generations):
            # Evaluate fitness of current population
            fitness_values = [self.fitness_function(chromosome) for chromosome in self.population]

            # Encontrar al mejor individuo de la generación actual
            current_best_index = np.argmax(fitness_values)
            current_best_fitness = fitness_values[current_best_index]

            # Comparar con el mejor histórico y actualizar si se encuentra uno nuevo
            if current_best_fitness > self.best_fitness_so_far:
                self.best_fitness_so_far = current_best_fitness
                self.best_chromosome_so_far = self.population[current_best_index]
                print(f"Generación {generation}: Nuevo mejor fitness = {self.best_fitness_so_far}")
            
            # Select parents
            selected_population = selection(self.population, fitness_values)
            
            # Apply crossover
            offspring = crossover(selected_population)
            
            # Apply mutation
            mutated_offspring = mutation(offspring, mutation_rate)
            
            # Update population
            self.population = mutated_offspring
            self.population[0] = self.best_chromosome_so_far
# Usage Example
population_size = 3
chromosome_length = num_features
num_generations = 5
mutation_rate = 0.01
genetic_algorithm = GeneticAlgorithm(population_size, chromosome_length, fitness_function)
genetic_algorithm.evolve(num_generations, mutation_rate)

print("\n--- Mejor Resultado Encontrado (con Elitismo) ---")
print(f"Mejor Score de Aptitud: {genetic_algorithm.best_fitness_so_far}")
print(f"Mejor Cromosoma (Características): {genetic_algorithm.best_chromosome_so_far}")
print(f"Número de características seleccionadas: {np.sum(genetic_algorithm.best_chromosome_so_far)}")

/home/juan/miniconda3/envs/py310/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Probando con 13 características. F1-score obtenido: 0.7851466414380321


/home/juan/miniconda3/envs/py310/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Probando con 13 características. F1-score obtenido: 0.790570797855566


/home/juan/miniconda3/envs/py310/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

/home/juan/miniconda3/envs/py310/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Probando con 12 características. F1-score obtenido: 0.7933301797540208


/home/juan/miniconda3/envs/py310/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Probando con 12 características. F1-score obtenido: 0.7933301797540208
Probando con 12 características. F1-score obtenido: 0.7933301797540208

--- Mejor Resultado Encontrado (con Elitismo) ---
Mejor Score de Aptitud: 0.7933301797540208
Mejor Cromosoma (Características): [1 1 0 1 1 1 0 0 0 0 1 0 1 1 0 1 1 0 0 1 1]
Número de características seleccionadas: 12


/home/juan/miniconda3/envs/py310/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
